In [1]:
import pandas as pd
import numpy as np

## Data Dictionary

The training set consists of data from 2007, 2009, 2011, and 2013,<br>
while in the test set you are requested to predict the test results for 2008, 2010, 2012, and 2014.

 - **Id:** id of the record
<br>
 - **Date:** date that the WNV test is performed
<br>
 - **Address:** approximate address of the location of trap. This is used to send to the GeoCoder.
<br>
 - **Species:** the species of mosquitos
<br>
 - **Block:** block number of address
<br>
 - **Street:** street name
<br>
 - **Trap:** Id of the trap
<br>
 - **AddressNumberAndStreet:** approximate address returned from GeoCoder
<br>
 - **Latitude, Longitude:** Latitude and Longitude returned from GeoCoder
<br>
 - **AddressAccuracy:** accuracy returned from GeoCoder
<br>
 - **NumMosquitos:** number of mosquitoes caught in this trap
<br>
 - **WnvPresent:** whether West Nile Virus was present in these mosquitos. 1 means WNV is present, and 0 means not present. 

In [2]:
# Locating the dataset paths
train_path = "./datasets/train.csv"
test_path = "./datasets/test.csv"
weather_path = "./datasets/weather.csv"
spray_path = "./datasets/spray.csv"

In [3]:
# Reading them into notebook as pd dataframes
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
weather = pd.read_csv(weather_path)
spray = pd.read_csv(spray_path)

In [4]:
# Setting target value
labels = train.WnvPresent

In [5]:
print(train.shape)
print(test.shape)
print(weather.shape)
print(spray.shape)

(10506, 12)
(116293, 11)
(2944, 22)
(14835, 4)


Train and test set:
<br>
 - Longtitude and latitude is sufficient for acquiring location of the trap, therefore other address values are redundant. 
 - Id feature is irrelevant and will also be dropped.
 - Label values (WnvPresent) have been extracted and can be dropped from the train set.
 - NumMosquitos values are not present in the test set and will be dropped from the train set.

In [6]:
# Saved NumMosquitos for possible modelling purposes 
mozzies = train[['Date', 'Species', 'Trap', 'Latitude', 'Longitude', 'NumMosquitos']]

In [7]:
mozzies.groupby(['Date', 'Species', 'Trap', 'Latitude', 'Longitude']).sum()

NumMosquitos
Date       Species                Trap Latitude  Longitude               
2007-05-29 CULEX PIPIENS          T096 41.731922 -87.677512             1
           CULEX PIPIENS/RESTUANS T002 41.954690 -87.800991             1
                                  T015 41.974089 -87.824812             1
                                  T048 41.867108 -87.654224             1
                                  T050 41.919343 -87.694259             1
...                                                                   ...
2013-09-26 CULEX RESTUANS         T082 41.803423 -87.642984             2
                                  T102 41.750498 -87.605294             1
                                  T209 41.740641 -87.546587             1
                                  T220 41.963976 -87.691810             8
                                  T221 41.659112 -87.538693             5

[8475 rows x 1 columns]

In [8]:
# Feature Elimination for train/test dataset
train.drop(['Address', 'Block', 'Street', 'AddressNumberAndStreet', 'AddressAccuracy', 'NumMosquitos', 'WnvPresent'], axis= 1, inplace= True)
test.drop(['Id', 'Address', 'Block', 'Street', 'AddressNumberAndStreet', 'AddressAccuracy'], axis= 1, inplace= True)

In [9]:
# Changing dtype of 'Date' column to datetime
train.Date = pd.to_datetime(train.Date)
test.Date = pd.to_datetime(test.Date)

Weather set: 
 - Missing values are represented as 'M', '-'. These will be filled with None.
 - Trace values represented by 'T' will be replaced by 0.1.
 - CodeSum column contains multiple empty rows and the feature information repetetive of other columns.
 - All the values in 'Water1' are missing, the columns will be dropped.
 - All the values in 'Depth' are 0.0 and only 17 values in 'SnowFall' are not 0.0 out of 1472, therefore these columns will be dropped
 - Rows will be seperated by station number and merged using the mean values of the features from stn1 and stn2. 

In [10]:
# Feature Elemination for the weather dataset
weather.drop(['CodeSum', 'Water1', 'Depth', 'SnowFall'], axis= 1, inplace= True)

In [11]:
# Stripping all whitespace from values in weather dataframe
for e in weather.columns:
    if weather[e].dtype == 'object':
        weather[e] = weather[e].apply(lambda x: x.strip())

In [12]:
# Replaceing missing values with None
weather = weather.apply(lambda x: x.replace(to_replace= ['M', '-'], value= None))

In [13]:
# Replacing trace values with 0.1
weather = weather.apply(lambda x: x.replace(to_replace= ['T'], value= 0.1))

In [14]:
# Changing dtype of columns below to integer
weather[['Tavg', 'Depart', 'WetBulb', 'Heat', 'Cool', 'Sunrise', 'Sunset']] = weather[['Tavg', 'Depart', 'WetBulb', 'Heat', 'Cool', 'Sunrise', 'Sunset']].astype(int)

In [15]:
# Changing dtype of columns below to float
weather[['PrecipTotal', 'StnPressure', 'SeaLevel', 'AvgSpeed']] = weather[['PrecipTotal', 'StnPressure', 'SeaLevel', 'AvgSpeed']].astype(float)

In [16]:
# Changing dtype of 'Date' column to datetime
weather.Date = pd.to_datetime(weather.Date)

In [17]:
# Splitting weather data by station number
weather_stn1 = weather[weather['Station']==1]
weather_stn2 = weather[weather['Station']==2]
weather_stn1 = weather_stn1.drop('Station', axis=1)
weather_stn2 = weather_stn2.drop('Station', axis=1)
print(weather_stn1.shape)
print(weather_stn2.shape)

(1472, 17)
(1472, 17)


In [18]:
# Taking the mean of 2 station's data togather to form a new dataframe
weather_fin = pd.concat([weather_stn1, weather_stn2]).groupby('Date', as_index=False).mean()

Spray set:
<br>
 - A column indicating 1 if spraying occured on the date and location specified was created.

In [19]:
spray['Spray'] = 1

In [20]:
spray.Date = pd.to_datetime(spray.Date)

In [21]:
spray.drop('Time', axis= 1, inplace= True)

In [22]:
spray.head()

,Date,Latitude,Longitude,Spray
0,2011-08-29,42.391623,-88.089163,1
1,2011-08-29,42.391348,-88.089163,1
2,2011-08-29,42.391022,-88.089157,1
3,2011-08-29,42.390637,-88.089158,1
4,2011-08-29,42.390410,-88.088858,1


Merging:
(spray dataset has been excluded)
- train + weather dataframes
- test + weather dataframes

In [23]:
# Final train dataset
train_fin = train.merge(weather_fin, how= 'left', on= 'Date')

In [24]:
# Final test Dataset
test_fin = test.merge(weather_fin, how= 'left', on= 'Date')

### Note to group: 
The cells below display the information for:
<br><br>
**train_fin** - Final version of the training dataset
<br><br>
**test_fin** - Final version of the test dataset
<br><br>
**labels** - Target/y dataset

In [25]:
train_fin.head()

,Date,Species,Trap,Latitude,Longitude,Tmax,Tmin,Tavg,Depart,DewPoint,...,Heat,Cool,Sunrise,Sunset,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,2007-05-29,CULEX PIPIENS/RESTUANS,T002,41.954690,-87.800991,88.0,62.5,75.5,10.0,58.5,...,0.0,10.5,421.0,1917.0,0.0,29.415,30.1,5.8,17.0,6.95
1,2007-05-29,CULEX RESTUANS,T002,41.954690,-87.800991,88.0,62.5,75.5,10.0,58.5,...,0.0,10.5,421.0,1917.0,0.0,29.415,30.1,5.8,17.0,6.95
2,2007-05-29,CULEX RESTUANS,T007,41.994991,-87.769279,88.0,62.5,75.5,10.0,58.5,...,0.0,10.5,421.0,1917.0,0.0,29.415,30.1,5.8,17.0,6.95
3,2007-05-29,CULEX PIPIENS/RESTUANS,T015,41.974089,-87.824812,88.0,62.5,75.5,10.0,58.5,...,0.0,10.5,421.0,1917.0,0.0,29.415,30.1,5.8,17.0,6.95
4,2007-05-29,CULEX RESTUANS,T015,41.974089,-87.824812,88.0,62.5,75.5,10.0,58.5,...,0.0,10.5,421.0,1917.0,0.0,29.415,30.1,5.8,17.0,6.95


In [26]:
test_fin.head()

,Date,Species,Trap,Latitude,Longitude,Tmax,Tmin,Tavg,Depart,DewPoint,...,Heat,Cool,Sunrise,Sunset,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,2008-06-11,CULEX PIPIENS/RESTUANS,T002,41.95469,-87.800991,86.0,63.5,75.0,7.0,55.5,...,0.0,10.0,416.0,1926.0,0.0,29.31,29.98,9.15,18.0,10.2
1,2008-06-11,CULEX RESTUANS,T002,41.95469,-87.800991,86.0,63.5,75.0,7.0,55.5,...,0.0,10.0,416.0,1926.0,0.0,29.31,29.98,9.15,18.0,10.2
2,2008-06-11,CULEX PIPIENS,T002,41.95469,-87.800991,86.0,63.5,75.0,7.0,55.5,...,0.0,10.0,416.0,1926.0,0.0,29.31,29.98,9.15,18.0,10.2
3,2008-06-11,CULEX SALINARIUS,T002,41.95469,-87.800991,86.0,63.5,75.0,7.0,55.5,...,0.0,10.0,416.0,1926.0,0.0,29.31,29.98,9.15,18.0,10.2
4,2008-06-11,CULEX TERRITANS,T002,41.95469,-87.800991,86.0,63.5,75.0,7.0,55.5,...,0.0,10.0,416.0,1926.0,0.0,29.31,29.98,9.15,18.0,10.2


In [31]:
labels.head()

0    0
1    0
2    0
3    0
4    0
Name: WnvPresent, dtype: int64

In [27]:
train_fin.columns

Index(['Date', 'Species', 'Trap', 'Latitude', 'Longitude', 'Tmax', 'Tmin',
       'Tavg', 'Depart', 'DewPoint', 'WetBulb', 'Heat', 'Cool', 'Sunrise',
       'Sunset', 'PrecipTotal', 'StnPressure', 'SeaLevel', 'ResultSpeed',
       'ResultDir', 'AvgSpeed'],
      dtype='object')

In [28]:
test_fin.columns

Index(['Date', 'Species', 'Trap', 'Latitude', 'Longitude', 'Tmax', 'Tmin',
       'Tavg', 'Depart', 'DewPoint', 'WetBulb', 'Heat', 'Cool', 'Sunrise',
       'Sunset', 'PrecipTotal', 'StnPressure', 'SeaLevel', 'ResultSpeed',
       'ResultDir', 'AvgSpeed'],
      dtype='object')

In [29]:
train_fin.dtypes

Date           datetime64[ns]
Species                object
Trap                   object
Latitude              float64
Longitude             float64
Tmax                  float64
Tmin                  float64
Tavg                  float64
Depart                float64
DewPoint              float64
WetBulb               float64
Heat                  float64
Cool                  float64
Sunrise               float64
Sunset                float64
PrecipTotal           float64
StnPressure           float64
SeaLevel              float64
ResultSpeed           float64
ResultDir             float64
AvgSpeed              float64
dtype: object

In [30]:
test_fin.dtypes

Date           datetime64[ns]
Species                object
Trap                   object
Latitude              float64
Longitude             float64
Tmax                  float64
Tmin                  float64
Tavg                  float64
Depart                float64
DewPoint              float64
WetBulb               float64
Heat                  float64
Cool                  float64
Sunrise               float64
Sunset                float64
PrecipTotal           float64
StnPressure           float64
SeaLevel              float64
ResultSpeed           float64
ResultDir             float64
AvgSpeed              float64
dtype: object